In [1]:
import pathlib
import sys

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy.ndimage
import scipy.signal
import skimage
import skimage.io as io
import skimage.morphology
import skimage.segmentation
from centrosome.cpmorphology import fixup_scipy_ndimage_result as fix
from scipy.linalg import lstsq

sys.path.append("../featurization")
import numpy
from area_size_shape import calulate_surface_area, measure_3D_area_size_shape
from colocalization import calculate_3D_colocalization
from data_writer import organize_featurization_data
from granularity import measure_3D_granularity
from intensity import measure_3D_intensity
from loading_classes import Featurization, ImageSetLoader, ObjectLoader
from neighbors import measure_3D_number_of_neighbors
from texture import measure_3D_texture

In [2]:
channel_mapping = {
    "nuclei": "405",
    "AGP": "488",
    "ER": "555",
    "Mito": "640",
    "BF": "TRANS",
    "nuclei_mask": "nuclei_",
    "cell_mask": "cell_",
    "cytoplasm_mask": "cytoplasm_",
    "organoid_mask": "organoid_",
}

In [3]:
image_set_path = pathlib.Path("../../data/NF0014/cellprofiler/C4-2/")

## Testing

In [4]:
image_set_loader = ImageSetLoader(
    image_set_path=image_set_path,
    spacing=(1, 0.1, 0.1),
    channel_mapping=channel_mapping,
)
image_set_loader.retrieve_image_attributes()
print(image_set_loader.image_name)
print(image_set_loader.unique_compartment_objects.keys())
print(image_set_loader.compartments)

['nuclei', 'AGP', 'ER', 'Mito', 'BF']
dict_keys(['cell_mask', 'cytoplasm_mask', 'nuclei_mask', 'organoid_mask'])
['cell_mask', 'cytoplasm_mask', 'nuclei_mask', 'organoid_mask']


In [ ]:
objects_loader = ObjectLoader(
    image=image_set_loader.get_image("nuclei"),
    label_image=image_set_loader.get_image("nuclei_mask"),
    channel_name="nuclei",
    compartment_name="nuclei",
    label_index=4,
)

NameError: name 'ObjectLoader' is not defined

In [ ]:
featurization = Featurization(
    image_set_loader=image_set_loader,
    object_loader=objects_loader,
    neighbors_distance_threshold=10,
)
features = featurization.get_features()

INFO:root:Calculated Area Size Shape features for nuclei 4


In [ ]:
compartment = "Nuclei"
channel = "DNA"
label_index = 4

In [ ]:
updated_features = {}
for key, value in features.items():
    if isinstance(value, dict):
        for sub_key, sub_value in value.items():
            updated_features[f"{compartment}_{channel}_{key}_{sub_key}"] = sub_value
    else:
        updated_features[f"{compartment}_{channel}_{key}_nosubfeature"] = value
updated_features

{'Nuclei_DNA_area.size.shape_VOLUME': 88442.0,
 'Nuclei_DNA_area.size.shape_SURFACE.AREA': 597.6736509775509,
 'Nuclei_DNA_area.size.shape_MAJOR.AXIS.LENGTH': 156.89838982543267,
 'Nuclei_DNA_area.size.shape_MINOR.AXIS.LENGTH': 12.31554866910244,
 'Nuclei_DNA_area.size.shape_CENTER.X': 742.1053232627032,
 'Nuclei_DNA_area.size.shape_CENTER.Y': 386.92353180615544,
 'Nuclei_DNA_area.size.shape_CENTER.Z': 5.3141493860383076,
 'Nuclei_DNA_area.size.shape_BBOX.VOLUME': 131130.0,
 'Nuclei_DNA_area.size.shape_MIN.X': 671,
 'Nuclei_DNA_area.size.shape_MAX.X': 812,
 'Nuclei_DNA_area.size.shape_MIN.Y': 343,
 'Nuclei_DNA_area.size.shape_MAX.Y': 436,
 'Nuclei_DNA_area.size.shape_MIN.Z': 1,
 'Nuclei_DNA_area.size.shape_MAX.Z': 11,
 'Nuclei_DNA_area.size.shape_EXTENT': 0.6744604590864028,
 'Nuclei_DNA_area.size.shape_EULER.NUMBER': 1,
 'Nuclei_DNA_area.size.shape_EQUIVALENT.DIAMETER': 55.2781259276932,
 'Nuclei_DNA_granularity_Granularity.1.nuclei': 100.0,
 'Nuclei_DNA_granularity_Granularity.2.nucl

In [ ]:
# make a df out of the features
df = pd.DataFrame(updated_features, index=[0])
# get the index of the object being measured
df.insert(0, "object_index", label_index)
df.insert(0, "image_set", image_set_loader.image_set_name)
df

,image_set,object_index,Nuclei_DNA_area.size.shape_VOLUME,Nuclei_DNA_area.size.shape_SURFACE.AREA,Nuclei_DNA_area.size.shape_MAJOR.AXIS.LENGTH,Nuclei_DNA_area.size.shape_MINOR.AXIS.LENGTH,Nuclei_DNA_area.size.shape_CENTER.X,Nuclei_DNA_area.size.shape_CENTER.Y,Nuclei_DNA_area.size.shape_CENTER.Z,Nuclei_DNA_area.size.shape_BBOX.VOLUME,...,Nuclei_DNA_intensity_mass.displacement,Nuclei_DNA_intensity_mad.intensity,Nuclei_DNA_intensity_edge.count,Nuclei_DNA_intensity_integrated.intensity.edge,Nuclei_DNA_intensity_mean.intensity.edge,Nuclei_DNA_intensity_std.intensity.edge,Nuclei_DNA_intensity_min.intensity.edge,Nuclei_DNA_intensity_max.intensity.edge,Nuclei_DNA_neighbors_10_nosubfeature,Nuclei_DNA_neighbors.adjacent_nosubfeature
0,C4-2,4,88442.0,597.673651,156.89839,12.315549,742.105323,386.923532,5.314149,131130.0,...,1028.820369,12316.623861,6498,38466019,5919.670514,6452.293137,0,24562,2,2


In [ ]:
organize_featurization_data(
    features=features,
    compartment=compartment,
    channel=channel,
    label_index=label_index,
    image_set_name=image_set_loader.image_set_name,
    output=True,
    # output_path=pathlib.Path("output.csv"),
    output_path=pathlib.Path("output.parquet"),
)

PosixPath('output.parquet')

## Put it all together

In [4]:
image_set_loader = ImageSetLoader(
    image_set_path=image_set_path,
    spacing=(1, 0.1, 0.1),
    channel_mapping=channel_mapping,
)
image_set_loader.retrieve_image_attributes()
image_set_loader.unique_objects.keys()

dict_keys(['cell_mask', 'cytoplasm_mask', 'nuclei_mask', 'organoid_mask'])

In [ ]:
# iterate through each unique object in each compartment for each channel image

for channel_image in image_set_loader.image_names:
    for compartment in image_set_loader.compartments:
        for unique_object_index in image_set_loader.unique_objects[compartment][1:3]:
            print(
                f"Channel: {channel_image}, Compartment: {compartment}, Object: {unique_object_index}"
            )
            # initialize object loader for each unique object
            objects_loader = ObjectLoader(
                image=image_set_loader.get_image(channel_image),
                label_image=image_set_loader.get_image(compartment),
                channel_name=channel_image,
                compartment_name=compartment,
                label_index=unique_object_index,
            )
            # featurize the object
            featurization = Featurization(
                image_set_loader=image_set_loader,
                object_loader=objects_loader,
                neighbors_distance_threshold=10,
            )

            features = featurization.get_features()
            # write the features to a csv file
            organize_featurization_data(
                features=features,
                channel=channel_image,
                compartment=compartment,
                label_index=unique_object_index,
                image_set_name=image_set_loader.image_set_name,
                return_df=False,
                output=True,
                output_path=pathlib.Path(
                    f"../profiles_features/{image_set_loader.image_set_name}_{channel_image}_{compartment}_{unique_object_index}.parquet"
                ),
            )

Channel: nuclei, Compartment: cell_mask, Object: 2


INFO:root:Calculated Area Size Shape features for cell_mask 2
INFO:root:Calculated Granularity features for cell_mask 2
INFO:root:Calculated Neighbors features for cell_mask 2
INFO:root:Calculated Adjacent Neighbors features for cell_mask 2


Channel: nuclei, Compartment: cell_mask, Object: 4


INFO:root:Calculated Area Size Shape features for cell_mask 4
/home/lippincm/miniforge3/envs/GFF_featurization/lib/python3.12/site-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/lippincm/miniforge3/envs/GFF_featurization/lib/python3.12/site-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
INFO:root:Calculated Granularity features for cell_mask 4
INFO:root:Calculated Neighbors features for cell_mask 4
INFO:root:Calculated Adjacent Neighbors features for cell_mask 4


Channel: nuclei, Compartment: cytoplasm_mask, Object: 2


...

INFO:root:Calculated Area Size Shape features for cell_mask 2
INFO:root:Calculated Granularity features for cell_mask 2
INFO:root:Calculated Neighbors features for cell_mask 2
INFO:root:Calculated Adjacent Neighbors features for cell_mask 2


Channel: ER, Compartment: cell_mask, Object: 4


INFO:root:Calculated Area Size Shape features for cell_mask 4
/home/lippincm/miniforge3/envs/GFF_featurization/lib/python3.12/site-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/lippincm/miniforge3/envs/GFF_featurization/lib/python3.12/site-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
INFO:root:Calculated Granularity features for cell_mask 4
INFO:root:Calculated Neighbors features for cell_mask 4
INFO:root:Calculated Adjacent Neighbors features for cell_mask 4


Channel: ER, Compartment: cytoplasm_mask, Object: 2


INFO:root:Calculated Area Size Shape features for cytoplasm_mask 2
